<a href="https://colab.research.google.com/github/roxacarv/sistemaderecomendacao_mal/blob/master/Manipula%C3%A7%C3%A3o_de_Dados_de_Avalia%C3%A7%C3%B5es_do_MAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Recomendação de Animes usando KNN

## Importando bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

##Importando dataset de animes

In [ ]:
animes = pd.read_csv("drive/MyDrive/animal_mal/animes.csv", encoding="latin-1", error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
animes

## Importando dataset de perfis de usuários

In [ ]:
profiles = pd.read_csv("drive/MyDrive/animal_mal/profiles.csv", encoding="latin-1", error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Dando um ID numérico para cada usuário porque o dataset de perfis não traz o id definido pelo MAL.

le.fit(profiles["profile"])
profiles["profile"]=le.transform(profiles["profile"])
le_profile_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

## Importando dataset de reviews dos usuários

In [ ]:
reviews = pd.read_csv("drive/MyDrive/animal_mal/reviews.csv", encoding="latin-1", error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Dando um ID numérico para cada usuário porque o dataset de perfis não traz o id definido pelo MAL.

le.fit(reviews["profile"])
reviews["profile"]=le.transform(reviews["profile"])
le_profile_mapping_reviews = dict(zip(le.classes_, le.transform(le.classes_)))

# Pré-processamento de dados


In [ ]:
# Removendo colunas desnecessárias de cada dataset
animes_df = animes[["uid", "title", "genre", "aired", "episodes", "popularity", "score"]]
animes_df.rename(columns = {"uid": "anime_uid"}, inplace=True)
profiles_df = profiles[["profile", "gender", "favorites_anime"]]
profiles_df.rename(columns = {"favorites_anime": "favorites"}, inplace=True)
reviews_df = reviews[["profile", "anime_uid", "score"]]
reviews_df.rename(columns = {"score": "user_score"}, inplace=True)
reviews_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,profile,anime_uid,user_score
0,6837,34096,8
1,32404,34599,10
2,44549,28891,7
3,35005,2904,9
4,31031,4181,10
...,...,...,...
192107,28456,1281,9
192108,24458,1281,9
192109,1474,1281,3
192110,999,1281,10


In [ ]:
reviews_df["profile"].value_counts()

24591    762
25673    743
38977    648
15305    566
36106    491
        ... 
38710      1
24995      1
19297      1
18191      1
44943      1
Name: profile, Length: 47885, dtype: int64

In [ ]:
x = reviews_df["profile"].value_counts() > 10

In [ ]:
x

24591     True
25673     True
38977     True
15305     True
36106     True
         ...  
38710    False
24995    False
19297    False
18191    False
44943    False
Name: profile, Length: 47885, dtype: bool

In [ ]:
y = x[x].index
print(y.shape)

(2854,)


In [ ]:
reviews_df = reviews_df[reviews_df["profile"].isin(y)]

In [ ]:
reviews_and_animes = reviews_df.merge(animes_df, on="anime_uid")
reviews_and_animes.head()

,profile,anime_uid,user_score,title,genre,aired,episodes,popularity,score
0,6837,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
1,6837,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
2,20650,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
3,20650,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94
4,27090,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94


In [ ]:
animes_reviews_times = reviews_and_animes.groupby("title")["score"].count().reset_index()

In [ ]:
animes_reviews_times.rename(columns = {"score": "times_score"}, inplace=True)
animes_reviews_times

,title,times_score
0,"""0""",2
1,"""Bungaku Shoujo"" Memoire",2
2,"""Bungaku Shoujo"" Movie",12
3,"""Calpis"" Hakkou Monogatari",2
4,"""Eiji""",1
...,...,...
6535,xxxHOLiC Kei,16
6536,xxxHOLiC Movie: Manatsu no Yoru no Yume,16
6537,xxxHOLiC Rou,4
6538,ÄlDLIVE,4


In [ ]:
f_reviews = reviews_and_animes.merge(animes_reviews_times, on="title")
f_reviews

,profile,anime_uid,user_score,title,genre,aired,episodes,popularity,score,times_score
0,6837,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94,24
1,6837,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94,24
2,20650,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94,24
3,20650,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94,24
4,27090,34096,8,Gintama.,"['Action', 'Comedy', 'Historical', 'Parody', '...","Jan 9, 2017 to Mar 27, 2017",12.0,800,8.94,24
...,...,...,...,...,...,...,...,...,...,...
130379,6186,10033,8,Toriko,"['Action', 'Adventure', 'Comedy', 'Fantasy', '...","Apr 3, 2011 to Mar 30, 2014",147.0,1267,7.65,4
130380,34957,9751,8,Strike Witches Movie,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,2464,7.65,4
130381,43593,9751,8,Strike Witches Movie,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,2464,7.65,4
130382,2270,9751,10,Strike Witches Movie,"['Action', 'Ecchi', 'Magic', 'Military', 'Sci-...","Mar 17, 2012",1.0,2464,7.65,4


In [ ]:
final_reviews = f_reviews[f_reviews["times_score"] > 10]
final_reviews.shape

(114798, 10)

In [ ]:
final_reviews.drop_duplicates(["profile", "title"], inplace=True)
final_reviews.shape

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(40954, 10)

In [ ]:
animes_transposition = final_reviews.pivot_table(columns="profile", index="title", values="user_score")

In [ ]:
animes_transposition

profile,5,33,38,50,74,82,95,98,103,126,...,47682,47686,47728,47740,47769,47791,47793,47796,47816,47821
title,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15 Bishoujo Hyouryuuki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ef: A Tale of Melodies.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ef: A Tale of Memories.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xxxHOLiC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
animes_transposition.fillna(0, inplace=True)
animes_transposition

profile,5,33,38,50,74,82,95,98,103,126,...,47682,47686,47728,47740,47769,47791,47793,47796,47816,47821
title,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11eyes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Bishoujo Hyouryuuki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ef: A Tale of Melodies.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ef: A Tale of Memories.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xxxHOLiC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix

animes_sparse = csr_matrix(animes_transposition)

In [ ]:
knn_model = NearestNeighbors(algorithm="brute")
knn_model.fit(animes_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
distances, suggestions = knn_model.kneighbors(animes_transposition.iloc[0, :].values.reshape(1, -1))

In [ ]:
animes_transposition.head()

profile,5,33,38,50,74,82,95,98,103,126,...,47682,47686,47728,47740,47769,47791,47793,47796,47816,47821
title,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11eyes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Bishoujo Hyouryuuki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in range(len(suggestions)):
  print(animes_transposition.index[suggestions[i]])

Index(['"Bungaku Shoujo" Movie', 'Bronze: Zetsuai Since 1989', 'Nami',
       'Dyogrammaton', 'IâCan'],
      dtype='object', name='title')
